***

TEST셋 사용
* 시간별 평균 대입 (done)
* 일별 시간별 평균 대입
* 월별 일별 시간별 평균 대입

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, timedelta # 시간 데이터 처리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델

In [2]:
test = pd.read_csv("data/test.csv", sep = ",")
submission = pd.read_csv("data/submission.csv")

In [3]:
# object 타입의 'Time'열을 datetime 타입으로 변환
test['Time'] = pd.to_datetime(test.Time)

# 'Time'열을 데이터프레임의 인덱스로 설정
test = test.set_index('Time')

In [4]:
# index에서 month, date, hour을 각각 뽑아내어 새로운 칼럼으로 설정
test['month'] = test.index.month
test['date'] = test.index.day
test['hour'] = test.index.hour

In [5]:
test.head()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X266,X20,X443,X347,X75,X107,X230,month,date,hour
Time,,,,,,,,,,,,,,,,,,,,,
2017-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.275,0.021,NaN,7,1,0
2017-07-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.222,0.021,NaN,7,1,1
2017-07-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.237,0.021,NaN,7,1,2
2017-07-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.229,0.020,NaN,7,1,3
2017-07-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.202,0.021,NaN,7,1,4


In [6]:
# 각 hour별로 groupby한 후 mean() 계산
hourly = test.groupby('hour').mean()

# c = 미터ID리스트 (즉 위 데이터프레임의 칼럼 level) (후에 for loop에서 사용하기 위해)
c = hourly.columns

# hourly 데이터프레임과 c 확인
hourly[c[0]][1]

1.1861232876712327

In [8]:
# NaN 값인 셀들에 해당 미터ID의 시간별 평균 대입

test2 = pd.DataFrame()   # 새로운 데이터프레임 생성
n=0  # 현재 칼럼카운트를 보여주기 위한 변수

# 미터ID 리스트 c에서 1개씩 for loop
for i in c:  
    n +=1   # 카운트 
    li = test[test[i].isnull()].index   # li: test데이터프레임의 특정 열(i)에서 Null인 셀 index추출 
    df = pd.DataFrame(test[i])    # df: test의 i열을 별개 데이터프레임으로 생성

    print("%d. current column: %s, NaN count: %d" %(n, i, len(li)))   
    for t in li:    #  li(Null 인덱스)를 사용해 for loop
        df.loc[t] = hourly[i][t.hour]  # null값에 t.hour 평균 대입
    test2 = pd.concat([test2,df], axis=1)   # 평균 대치가 끝난 df를 test2에 concat해준다. 

1. current column: X26, NaN count: 7003
2. current column: X303, NaN count: 1852
3. current column: X241, NaN count: 1894
4. current column: X435, NaN count: 2008
5. current column: X402, NaN count: 2003
6. current column: X352, NaN count: 1903
7. current column: X305, NaN count: 2021
8. current column: X350, NaN count: 1926
9. current column: X326, NaN count: 2037
10. current column: X299, NaN count: 2017
11. current column: X160, NaN count: 2088
12. current column: X125, NaN count: 0
13. current column: X211, NaN count: 1905
14. current column: X349, NaN count: 2016
15. current column: X407, NaN count: 2006
16. current column: X330, NaN count: 1985
17. current column: X132, NaN count: 1886
18. current column: X231, NaN count: 2094
19. current column: X225, NaN count: 2065
20. current column: X16, NaN count: 7003
21. current column: X286, NaN count: 2040
22. current column: X354, NaN count: 1969
23. current column: X353, NaN count: 2024
24. current column: X76, NaN count: 0
25. curren

198. current column: X75, NaN count: 0
199. current column: X107, NaN count: 0
200. current column: X230, NaN count: 2109
201. current column: month, NaN count: 0
202. current column: date, NaN count: 0


In [9]:
test2.tail()

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X223,X266,X20,X443,X347,X75,X107,X230,month,date
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-30 19:00:00,1.255,0.175,0.430,0.429,0.169,0.739,0.573,0.341,0.127,0.286,...,0.479,0.858,11.546,0.428,0.134,3.519,0.027,0.286,6,30
2018-06-30 20:00:00,1.596,1.692,0.332,0.366,0.159,0.255,0.501,0.281,0.126,0.230,...,0.725,0.816,11.591,0.413,0.206,3.331,0.027,0.303,6,30
2018-06-30 21:00:00,1.379,0.883,0.435,0.497,0.166,0.233,0.634,0.401,0.439,0.346,...,0.426,0.659,10.766,0.466,0.324,3.416,0.026,0.253,6,30
2018-06-30 22:00:00,1.000,0.539,0.268,0.459,0.463,0.454,0.676,0.543,0.737,0.466,...,0.353,0.462,0.312,0.473,0.402,4.263,0.027,0.253,6,30
2018-06-30 23:00:00,0.976,0.481,0.220,0.418,0.646,0.419,0.680,0.499,0.304,0.405,...,0.349,0.792,0.309,0.519,0.334,0.784,0.027,0.310,6,30


In [10]:
test3 = test2.drop(columns=['month', 'date'])   # month, date 칼럼을 쓰지 않았으므로 그냥 지운다. (처음부터 안 만들어도 됨ㅎ)

In [11]:
test3.to_csv("data/시간별평균대치.csv")

<br>

Arima 모델 실행

In [72]:
# 이하는 dacon 중급 baseline code 발췌(?)

time = []; place_id = []; target = []  

# test3 데이터프레임을 time인덱스, 미터ID(place_id)칼럼, 측정값(target)칼럼으로 정비
for i in test3.columns:
    for j in range(len(test3)):
        place_id.append(i)
        time.append(test3.index[j])
        target.append(test3[i].iloc[j])
new_df = pd.DataFrame({'place_id': place_id, 'time':time, 'target': target})
new_df = new_df.dropna()
new_df = new_df.set_index('time')
new_df.head()

,place_id,target
time,,
2017-07-01 00:00:00,X26,1.187315
2017-07-01 01:00:00,X26,1.186123
2017-07-01 02:00:00,X26,1.168301
2017-07-01 03:00:00,X26,1.163425
2017-07-01 04:00:00,X26,1.398466


In [73]:
import itertools

In [74]:
# ARIMA에서 사용할 최적의 파라미터 선정을 위해 다양한 경우의 pdq 생성
p=d=q = range(0,2)
pdq = list(itertools.product(p,d,q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [75]:
# ARIMA 최적의 파라미터 선정 함수
def get_optimal_params(y):
    param_dict = {}
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [76]:
# agg라는 dictionary를 생성하여, 각 미터ID를 key로, 예측시간 및 값 리스트를 value로 저장
agg={}

for key in new_df['place_id'].unique(): # 미터ID리스트를 unique()함수로 추출, for loop
    temp = new_df.loc[new_df['place_id']==key] # new_df에서 key와 일치하는 place_id를 가지는 부분을 temp에 할당
    temp_1h=temp.resample('1h').sum() # 1시간 단위 resampling(일종의 timeseries압축)
    temp_1day=temp.resample('D').sum() # 1일 단위 resampling

    # 1 시간별 예측
    model = ARIMA(temp_1h['target'],  # target 을 추측하고자 함
                  order=get_optimal_params(temp_1h['target'])) # AIC를 최소화하는 최적의 파라미터 
    results_ARIMA = model.fit(disp=-1)  
    fcst = results_ARIMA.forecast(24) # 24시간 예측

    a = pd.DataFrame() # a: 예측값을 담을 데이터프레임 생성
    
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[0][i]] # column명 지정 및 예측값 대입

        
    # 1 일별 예측
    model = ARIMA(temp_1day['target'], 
                  order=get_optimal_params(temp_1day['target'])) 
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(10) # 10일을 예측

    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=[fcst[0][i]] # column명 지정 및 예측값 대입
    
    
    # 월별 예측
    # 일별 예측의 sum 사용
    fcst = results_ARIMA.forecast(153)
    
    # column명을 지정 및 예측값 대입
    a['X2018_7_m'] = [np.sum(fcst[0][:31])] # 7월 
    a['X2018_8_m'] = [np.sum(fcst[0][31:62])] # 8월
    a['X2018_9_m'] = [np.sum(fcst[0][62:92])] # 9월
    a['X2018_10_m'] = [np.sum(fcst[0][92:123])] # 10월
    a['X2018_11_m'] = [np.sum(fcst[0][123:153])] # 11월
    
    # a 에 meter_id를 현재 예측하고 있는 열의 id(key)로 대입
    a['meter_id'] = key 
    
    # agg{ 미터ID: 시간별(subimssion.columns는 예측시간칼럼들) a의 예측값}
    agg[key] = a[submission.columns.tolist()]
    print(key)
print('---- Modeling Done ----')

X26


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X303
X241
X435
X402
X352
X305
X350
X326
X299
X160
X125


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X211
X349
X407
X330
X132
X231
X225
X16
X286
X354
X353
X76
X255
X106
X393
X157
X323
X130
X269
X94
X172
X289
X216
X123
X398
X233
X214
X270
X371
X200
X111
X163
X364
X378
X271
X462
X427


...

X368
X282
X426


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X47
X192
X283


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X329
X223
X266
X20
X443
X347
X75
X107
X230
---- Modeling Done ----


In [83]:
# 최종 output 마무리 및 확인
output1 = pd.concat(agg, ignore_index = False)
output2 = output1.reset_index().drop(['level_0', 'level_1'], axis=1)
output2['id'] = output2['meter_id'].str.replace('X','').astype(int)
output2 = output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
output2.head()

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X5,6.853865,6.662421,6.535229,6.450732,6.394607,6.357335,6.332590,6.316171,6.305283,...,151.158682,151.165311,151.188368,151.219685,151.255155,4700.315536,4734.280821,4617.861104,4809.298795,4690.459144
1,X7,1.258811,1.275497,1.289773,1.301985,1.312433,1.321371,1.329018,1.335560,1.341157,...,40.379061,40.218087,40.145692,40.118418,40.114128,1268.281828,1265.220143,1242.315902,1302.232722,1278.134528
2,X10,2.166754,2.297002,2.408626,2.504290,2.586275,2.656537,2.716753,2.768359,2.812586,...,73.933546,73.933546,73.933546,73.933546,73.933546,2303.938637,2291.939922,2218.006376,2291.939922,2218.006376
3,X16,3.294517,3.381436,3.454943,3.517108,3.569681,3.614141,3.651741,3.683540,3.710432,...,112.487417,112.467445,112.489421,112.529645,112.577808,3514.788138,3552.022654,3487.105398,3654.661835,3586.433638
4,X18,2.500810,2.510078,2.517351,2.523060,2.527539,2.531054,2.533812,2.535976,2.537673,...,61.992220,61.938487,61.909200,61.893237,61.884536,1923.494139,1918.097490,1856.223377,1918.097490,1856.223377


In [85]:
output2.to_csv('result/01.시간별평균.csv', index=False)